# SlideRule Voila Demo

SlideRule is an on-demand science data processing service that runs in the cloud and responds to REST API calls to process and return science results.

This web page demonstrates a simple use of SlideRule to return elevations within a small region of interest using processing parameters specified at the time of the request.  The web page is implemented inside a Jupyter Notebook using SlideRule's Python client, and is statically served using Voila.  The source code for what you see here can be found at: https://github.com/ICESat2-SlideRule/sliderule-python/blob/main/examples/voila_demo.ipynb. 

For more information on SlideRule, how to install it and use it for your own analysis applications, please see our website at [icesat2sliderule.org](http://icesat2sliderule.org).

In [1]:
# load the necessary packages
from sliderule import icesat2, ipysliderule, io
import ipywidgets as widgets
import geopandas
import logging
import warnings
import time

In [2]:
# turn off warnings for demo
warnings.filterwarnings('ignore')

In [3]:
# set the url for the sliderule service
# set the logging level
icesat2.init("icesat2sliderule.org", loglevel=logging.WARNING)

In [4]:
# create global variables
gdf = None
parms = None
results = []

In [5]:
# callback for handling results
def demoCB(resource, result, index, total):
    global gdf, parms, results
    print(f'SlideRule processing request... [{index}/{total}] {resource} returned {len(results)} elevations\r', end="")
    results.append(result)

# build and transmit requests to SlideRule
def runSlideRule():
    global gdf, parms, results 
    
    # sliderule asset and data release
    asset = SRwidgets.asset.value
    release = SRwidgets.release.value

    # build sliderule parameters using latest values from widget
    parms = {
        # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
        "srt": SRwidgets.surface_type.index,
        # length of ATL06-SR segment in meters
        "len": SRwidgets.length.value,
        # step distance for successive ATL06-SR segments in meters
        "res": SRwidgets.step.value,
        # confidence level for PE selection
        "cnf": SRwidgets.confidence.value,
        # ATL08 land surface classifications
        "atl08_class": list(SRwidgets.land_class.value),
        # maximum iterations, not including initial least-squares-fit selection
        "maxi": SRwidgets.iteration.value,
        # minimum along track spread
        "ats": SRwidgets.spread.value,
        # minimum PE count
        "cnt": SRwidgets.count.value,
        # minimum height of PE window in meters
        "H_min_win": SRwidgets.window.value,
        # maximum robust dispersion in meters
        "sigma_r_max": SRwidgets.sigma.value
    }

    # clear existing geodataframe
    results = []
    gdf = icesat2.__emptyframe()

    # for each region of interest
    for poly in m.regions:
        # add polygon from map to sliderule parameters
        parms["poly"] = poly 
        # make the request to the SlideRule (ATL06-SR) endpoint
        # and pass it the request parameters to request ATL06 Data
        icesat2.atl06p(parms, asset, callback=demoCB, version=release)

    gdf = geopandas.pd.concat(results)
    return gdf

### Step 1: Select Region of Interest

In [6]:
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet('Global')
m.map

Map(center=[39, -108], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### Step 2: Choose Processing Parameters and Run SlideRule

In [8]:
# button action
def on_button_clicked1(b):
    with output1:
        print(f'SlideRule processing request... initiated\r', end="")
        perf_start = time.perf_counter()
        gdf = runSlideRule()
        perf_duration = time.perf_counter() - perf_start
        print(f'SlideRule processing request... completed in {perf_duration:.3f} seconds; returned {gdf.shape[0]} records                                   ')
        m.GeoData(gdf, column_name="h_mean", cmap="viridis")

# create button to run sliderule
button1 = widgets.Button(description="Run SlideRule!")
output1 = widgets.Output()
button1.on_click(on_button_clicked1)
display(output1)
display(button1)

# display widgets for setting SlideRule parameters
SRwidgets = ipysliderule.widgets()
widgets.VBox([
    SRwidgets.surface_type,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.confidence,
    SRwidgets.land_class,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
])

Output()

Button(description='Run SlideRule!', style=ButtonStyle())

See http://icesat2sliderule.org/rtd/user_guide/ICESat-2.html#elevations for descriptions of each column

### Step 3: Explore Data Points and Refresh Plot

In [9]:
# refresh action
def on_refresh_clicked(b):
    global gdf
    with refresh_output:
        m.GeoData(gdf, column_name=SRwidgets.variable.value, cmap=SRwidgets.colormap)

# create button to display geodataframe        
refresh_button = widgets.Button(description="Refresh Plot")
refresh_output = widgets.Output()
refresh_button.on_click(on_refresh_clicked)
display(refresh_button, refresh_output)

Button(description='Refresh Plot', style=ButtonStyle())

Output()

In [10]:
widgets.VBox([
    SRwidgets.variable,
    SRwidgets.cmap,
    SRwidgets.reverse
])